In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('dataset1.csv')
X = df.iloc[:, :-1]
y = df.iloc[:, -1]
X.head()

,Arm length (m),Ball weight (kg),Ball radius (mm),Air temperature (deg C),Spring constant (N per m),Device weight (kg)
0,0.291903,0.340028,0.013823,18.0,545.348164,4.616101
1,0.247545,0.521061,0.390693,26.0,356.041022,5.863920
2,0.483827,0.451309,0.170141,11.0,522.387220,5.020725
3,0.216393,0.460698,0.101122,20.0,95.025757,4.586624
4,0.164084,0.391740,0.155177,11.0,104.242640,3.677411


In [3]:
X['Air temperature (deg C)'] += 273.15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/tmp/ipykernel_849184/699582764.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Air temperature (deg C)'] += 273.15


In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD, Adam
from keras.regularizers import l2, l1_l2, l1
import keras_tuner as kt

def model_builder(hp):
    model = Sequential()
    
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.0, max_value=0.5, default=0.0, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.0, max_value=0.5, default=0.0, step=0.1)
    hp_dropout_3 = hp.Float('dropout_3', min_value=0.0, max_value=0.5, default=0.0, step=0.1)
    
    hp_reg_1 = hp.Choice('l1', [1e-2, 1e-3, 1e-4])
    hp_reg_2 = hp.Choice('l2', [1e-2, 1e-3, 1e-4])
    hp_reg_3 = hp.Choice('l3', [1e-2, 1e-3, 1e-4])
    hp_reg_4 = hp.Choice('l4', [1e-2, 1e-3, 1e-4])
    hp_reg_5 = hp.Choice('l5', [1e-2, 1e-3, 1e-4])
    hp_reg_6 = hp.Choice('l6', [1e-2, 1e-3, 1e-4])
    
    model.add(Dense(64, activation='relu', activity_regularizer=(l1_l2(hp_reg_1, hp_reg_2))))
    model.add(Dropout(hp_dropout_1))
    model.add(Dense(128, activation='relu', activity_regularizer=(l1_l2(hp_reg_3, hp_reg_4))))
    model.add(Dropout(hp_dropout_2))
    model.add(Dense(256, activation='relu', activity_regularizer=(l1_l2(hp_reg_5, hp_reg_6))))
    model.add(Dropout(hp_dropout_3))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=Adam(learning_rate=3e-4), loss='binary_crossentropy', metrics=['acc'])
    
    return model

In [5]:
tuner = kt.Hyperband(
    model_builder,
    max_epochs=100,
    objective='val_acc',
    seed=52,
    executions_per_trial=2,
    directory='hyperband',
    project_name='coursework'
)

2021-11-15 13:53:18.608214: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-15 13:53:18.611914: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2021-11-15 13:53:18.611924: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-11-15 13:53:18.612785: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [6]:
es = EarlyStopping('val_acc', patience=100, mode='auto')
tuner.search(X_train, y_train, epochs=10000, validation_split=0.3, shuffle=True, callbacks=[es])

Trial 64 Complete [00h 00m 01s]
val_acc: 0.8833333253860474

Best val_acc So Far: 0.9135416746139526
Total elapsed time: 00h 01m 22s

Search: Running Trial #65

Hyperparameter    |Value             |Best Value So Far 
dropout_1         |0.4               |0.1               
dropout_2         |0                 |0.4               
dropout_3         |0.5               |0.2               
l1                |0.001             |0.001             
l2                |0.001             |0.001             
l3                |0.01              |0.0001            
l4                |0.01              |0.001             
l5                |0.001             |0.0001            
l6                |0.01              |0.01              
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |4                 |4                 
tuner/round       |0                 |0                 

Epoch 1/2
35/35 [=======================

KeyboardInterrupt: 

In [ ]:
tuner.get_best_hyperparameters(num_trials=1)[1].get('l1')